In [ ]:
import os
from collections import defaultdict

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil2 as teo
import utils
from rasterstats import zonal_stats

plt.style.use("ggplot")

In [ ]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

# TEOTIL2 Metals

**Note:** This notebook uses data from NIVA's JupyterHub and should be run there.

## Part 1: Spatial interpolation and regression

For TEOTIL2 Metals, we will assume that metals in freshwaters come from three possible sources:

 * Point inputs from industry, renseanlegg, fish farms etc.
 * Diffuse inputs from atmostpheric deposition
 * Diffuse inputs from geological weathering
 
Extending TEOTIL2 to include metals requires deriving two main parameters for each metal:

 1. **Export coefficients** to convert concentrations measured in mosses (assumed to reflect atmospheric deposition) into actual deposition inputs to water bodies, and 
 
 2. **Retention factors** for each catchment, describing how much of the material added from point, geological and atmospheric souces is retained in each catchment (and how much is transmitted downstream)
 
Initial development of TEOTIL2 Metals (documented [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/nope_metals_3.ipynb)) attempted to derive export coefficients by comparing the moss and geological datasets to water chemistry data from the **1995 "1000 Lakes" survey**. However, for most metals, the proportion of samples from the 1995 survey with values at or **below the limit of detection (LOD)** is very high, making it impossible to identify robust relationships. Development of TEOTIL2 Metals was therefore postponed for one year, in anticipation of a new "1000 Lakes" survey with lower LODs scheduled for autumn 2019.

The **2019 1000 Lakes** dataset provides a comprehensive and up-to-date picture of metal concentrations across Norway. This notebook repeats some of the preliminary analysis using the new dataset. The aims are to:

 1. Interpolate the moss and geochemical datasets onto a common grid for comparison
 
 2. For each of the "1000 Lakes" catchments, calculate "zonal statistics" summarising mean moss and geochemical concentrations
 
 3. Explore statistical relationships relating metal concentrations measured in lakes in 2019 to the moss and geochemistry datasets

## 1. Define common grid for spatial interpolation

In [ ]:
# Define co-ord system
crs = ccrs.AlbersEqualArea(
    central_longitude=15,
    central_latitude=65,
    false_easting=650000,
    false_northing=800000,
    standard_parallels=(55, 75),
)

# Set up grid
cell_size = 1000  # metres

# Extent
xmin = 0
xmax = 1300000
ymin = 0
ymax = 1600000
extent = [xmin, xmax, ymin, ymax]

# Set up grid for interpolation
# Shift by (cell_size/2) s.t. values are estimated for the centre of each grid square
gridx = np.arange(xmin, xmax, cell_size) + (cell_size / 2.0)
gridy = np.arange(ymin, ymax, cell_size) + (cell_size / 2.0)

# Get Natural Earth data
land_50m = cfeature.NaturalEarthFeature(
    category="cultural",
    name="admin_0_countries",
    scale="50m",
    edgecolor="black",
    facecolor=cfeature.COLORS["land"],
)

sea_50m = cfeature.NaturalEarthFeature(
    category="physical",
    name="ocean",
    scale="50m",
    edgecolor="none",
    facecolor=cfeature.COLORS["water"],
    alpha=1,
)

country_shp = cartopy.io.shapereader.natural_earth(
    resolution="50m", category="cultural", name="admin_0_countries"
)

## 2. Spatially interpolate moss data

Moss surveys are conducted every 5 years, but the number of sites sampled in recent years has been reduced substantially. The most comprehensive survey in terms of the number of data points and use if modern analytical methods (i.e. low LODs) seems to be from 2005. This dataset is used below. 

**Note:** Parameters used for the spatial interpolation algorithm below are taken from the preliminary analysis.

In [ ]:
# Read moss data
year = 2005
moss_par_list = ["As", "Cd", "Cr", "Cu", "Hg", "Ni", "Pb", "Zn"]
moss_xlsx = r"../data/metals/mosses/moss_data_tidied.xlsx"
moss_df = pd.read_excel(moss_xlsx, sheet_name="mosses_%s" % year)
moss_df[moss_df < 0] = 0

for par in moss_par_list:
    print(f"Processing: {year} {par.capitalize()}")
    par = par.lower()
    df = moss_df[["lat", "lon", par]].dropna(how="any")

    # Map (long, lat) to projected (x, y) and reformat to syntax required
    # by interp algorithms
    pts = crs.transform_points(
        src_crs=ccrs.PlateCarree(), x=df["lon"].values, y=df["lat"].values
    )[:, :2]

    # Get values to interpolate
    vals = df[par].values

    # IDW interpolation
    idw = nivapy.spatial.interp_idw(pts, vals, gridx, gridy, n_near=8, p=2)

    # Save output
    idw_path = f"../data/metals/mosses/moss_{par}_{year}_mgpkg_idw_n8_p2.tif"
    nivapy.spatial.array_to_gtiff(
        xmin,
        ymax,
        cell_size,
        idw_path,
        idw,
        crs.proj4_init,
        no_data_value=-9999,
        bit_depth="Float32",
    )

In [ ]:
# Setup fig
fig = plt.figure(figsize=(15, 15))

# Loop over data
for idx, par in enumerate(moss_par_list):
    # Read data
    gtiff_path = f"../data/metals/mosses/moss_{par.lower()}_{year}_mgpkg_idw_n8_p2.tif"
    data = nivapy.spatial.read_raster(gtiff_path, band_no=1)[0]

    # Plot
    ax = fig.add_subplot(3, 3, idx + 1, projection=crs)
    ax.set_extent(extent, crs=crs)
    ax.set_title("%s (%s)" % (par.capitalize(), year), fontsize=20)
    cax = ax.imshow(
        data,
        zorder=1,
        extent=extent,
        cmap="coolwarm",
        alpha=0.7,
        interpolation="none",
        vmax=np.nanpercentile(data, 95),
    )

    # Add colourbar
    cbar = fig.colorbar(cax)

    # Make Norway "hollow" so interpolated vaues are visible.
    # Make everywhere else opaque to mask wildly extrapolated values
    reader = cartopy.io.shapereader.Reader(country_shp)
    countries = reader.records()
    for country in countries:
        if country.attributes["NAME"] == "Norway":
            # Transparent
            ax.add_geometries(
                [country.geometry],
                ccrs.PlateCarree(),  # CRS of Natural Earth data
                facecolor="none",
                edgecolor="black",
                zorder=5,
            )
        else:
            # Opaque
            ax.add_geometries(
                [country.geometry],
                ccrs.PlateCarree(),  # CRS of Natural Earth data
                facecolor=cfeature.COLORS["land"],
                edgecolor="black",
                zorder=5,
            )

    # Add sea
    ax.add_feature(sea_50m, zorder=4)

plt.tight_layout()

# Save
out_path = f"../plots/metals/mosses/mosses_{year}_idw_n8_p2.png"
plt.savefig(out_path, dpi=300)

## 3. Spatially interpolate geochemistry data

The workflow below is taken from the preliminary analysis.

In [ ]:
# Read geo data
geo_xlsx = r"../data/metals/geochemistry/ngu_geochem_tidied.xlsx"
geo_df = pd.read_excel(geo_xlsx, sheet_name="nitric_acid")
del geo_df["date"], geo_df["from_depth"], geo_df["to_depth"]

# Loop over pars
geol_par_list = ["As", "Cr", "Cu", "Ni", "Pb", "Zn"]

for par in geol_par_list:
    print(f"Processing: {par}")

    par = par.lower()
    df = geo_df[["lat", "lon", par]].dropna(how="any")

    # Map (long, lat) to projected (x, y) and reformat to syntax required
    # by interp algorithms
    pts = crs.transform_points(
        src_crs=ccrs.PlateCarree(), x=df["lon"].values, y=df["lat"].values
    )[:, :2]

    # Get values to interpolate
    vals = df[par].values

    # IDW interpolation
    idw = nivapy.spatial.interp_idw(pts, vals, gridx, gridy, n_near=20, p=1)

    # Save output
    idw_path = f"../data/metals/geochemistry/geochem_{par}_mgpkg_idw_n20_p1.tif"
    nivapy.spatial.array_to_gtiff(
        xmin,
        ymax,
        cell_size,
        idw_path,
        idw,
        crs.proj4_init,
        no_data_value=-9999,
        bit_depth="Float32",
    )

In [ ]:
# Setup fig
fig = plt.figure(figsize=(15, 15))

# Loop over data
for idx, par in enumerate(geol_par_list):
    # Read data
    gtiff_path = (
        f"../data/metals/geochemistry/geochem_{par.lower()}_mgpkg_idw_n20_p1.tif"
    )
    data = nivapy.spatial.read_raster(gtiff_path, band_no=1)[0]

    # Plot
    ax = fig.add_subplot(3, 3, idx + 1, projection=crs)
    ax.set_extent(extent, crs=crs)
    ax.set_title("%s (%s)" % (par.capitalize(), year), fontsize=20)
    cax = ax.imshow(
        data,
        zorder=1,
        extent=extent,
        cmap="coolwarm",
        alpha=0.7,
        interpolation="none",
        vmax=np.nanpercentile(data, 95),
    )

    # Add colourbar
    cbar = fig.colorbar(cax)

    # Make Norway "hollow" so interpolated vaues are visible.
    # Make everywhere else opaque to mask wildly extrapolated values
    reader = cartopy.io.shapereader.Reader(country_shp)
    countries = reader.records()
    for country in countries:
        if country.attributes["NAME"] == "Norway":
            # Transparent
            ax.add_geometries(
                [country.geometry],
                ccrs.PlateCarree(),  # CRS of Natural Earth data
                facecolor="none",
                edgecolor="black",
                zorder=5,
            )
        else:
            # Opaque
            ax.add_geometries(
                [country.geometry],
                ccrs.PlateCarree(),  # CRS of Natural Earth data
                facecolor=cfeature.COLORS["land"],
                edgecolor="black",
                zorder=5,
            )

    # Add sea
    ax.add_feature(sea_50m, zorder=4)

plt.tight_layout()

# Save
out_path = f"../plots/metals/geochemistry/geochem_idw_n20_p1.png"
plt.savefig(out_path, dpi=300)

## 4. Process 1000 Lakes dataset

### 4.1 Read 2019 water chemistry

In [ ]:
wc_df = pd.read_csv(r"../data/metals/1k_lakes/1k_lakes_2019_metals_toc_ph.csv")
del wc_df["station_id"], wc_df["station_name"]
stn_df = pd.read_csv(r"../data/metals/1k_lakes/1k_stations.csv")
del stn_df["name"]
wc_df = pd.merge(stn_df, wc_df, how="inner", on="station_code")
wc_df.head()

### 4.2. Get catchment boundaries

The 1000 Lakes catchment boundaries are stored on NIVA's JupyterHub.

In [ ]:
# Show projects in database
with pd.option_context("display.max_colwidth", -1):
    proj_df = nivapy.da.select_jhub_projects(eng)
    display(proj_df)

In [ ]:
# Get catchment data for 1000 Lakes 2019
_, cat_gdf = nivapy.da.select_jhub_project_catchments([4], eng)

cat_gdf.head()

## 5. Zonal statistics

The code in this section summarises the moss and geochemical datasets for each of the 1000 Lakes catchments.

### 5.1. Moss data

In [ ]:
df_list = []
for par in moss_par_list:
    par = par.lower()
    ras_path = f"../data/metals/mosses/moss_{par.lower()}_{year}_mgpkg_idw_n8_p2.tif"

    # Temporary nearest-neighbour resampling to 250 m (as some cacthments are so small that
    # they do not contain any cell centres at 1000 m)
    data = nivapy.spatial.read_raster(ras_path, band_no=1)[0]
    data = nivapy.spatial.rebin_array(data, 4)  # 1000 m to 250 m
    tmp_path = "temp_ras.tif"
    nivapy.spatial.array_to_gtiff(
        xmin,
        ymax,
        250,
        tmp_path,
        data,
        crs.proj4_init,
        no_data_value=-9999,
        bit_depth="Float32",
    )

    # Zonal stats
    stats = zonal_stats(
        cat_gdf["geom"].to_crs(crs.proj4_init), tmp_path, stats=["mean"]
    )
    stats = pd.DataFrame(stats)
    stats.rename({"mean": f"moss_mean_{par}"}, inplace=True, axis="columns")
    df_list.append(stats)

    os.remove(tmp_path)

moss_stats = pd.concat(df_list, axis="columns")
moss_stats["station_code"] = cat_gdf["station_code"]

moss_stats.head()

### 5.2. Geochemistry data

In [ ]:
df_list = []
for par in geol_par_list:
    par = par.lower()
    ras_path = f"../data/metals/geochemistry/geochem_{par.lower()}_mgpkg_idw_n20_p1.tif"

    # Temporary nearest-neighbour resampling to 250 m (as some cacthments are so small that
    # they do not contain any cell centres at 1000 m)
    data = nivapy.spatial.read_raster(ras_path, band_no=1)[0]
    data = nivapy.spatial.rebin_array(data, 4)  # 1000 m to 250 m
    tmp_path = "temp_ras.tif"
    nivapy.spatial.array_to_gtiff(
        xmin,
        ymax,
        250,
        tmp_path,
        data,
        crs.proj4_init,
        no_data_value=-9999,
        bit_depth="Float32",
    )

    stats = zonal_stats(
        cat_gdf["geom"].to_crs(crs.proj4_init), tmp_path, stats=["mean"]
    )
    stats = pd.DataFrame(stats)
    stats.rename({"mean": f"geol_mean_{par}"}, inplace=True, axis="columns")
    df_list.append(stats)

    os.remove(tmp_path)

geol_stats = pd.concat(df_list, axis="columns")
geol_stats["station_code"] = cat_gdf["station_code"]

geol_stats.head()

## 6. Combine datasets

In [ ]:
# Link water chem, moss and geol datasets
stats = pd.merge(moss_stats, geol_stats, how="inner", on="station_code")
df = pd.merge(wc_df, stats, how="inner", on="station_code")

# Convert pH to [H+]
df["hp"] = 10 ** (-df["pH_"])

df.head()

## 7. Regression

Of the 8 metals of interest, only 6 have complete geochemistry data (Cd and Hg are missing). For each of these 6 parameters, the code below first produces scatter plots to hgihlight general relationships & obvious outliers, and then performs "best subsets" OLS regression to give a more quantitative assessment of any relationships present. The key question here is:

 * Can we use **only** the moss and geochemistry datasets as predictors of water quality

### 7.1. As

In [ ]:
par = "As"

cols = [
    f"{par}_µg/l",
    f"moss_mean_{par.lower()}",
    f"geol_mean_{par.lower()}",
    "TOC_mg C/l",
    "hp",
]
reg_df = df[cols].copy()

with sn.plotting_context("notebook", font_scale=1.5):
    sn.pairplot(
        reg_df,
        kind="reg",
        diag_kind="kde",
        plot_kws=dict(lowess=True, line_kws=dict(color="red")),
    )

In [ ]:
thresh = 1
resp_var = f"{par}_µg/l"
exp_vars = [f"moss_mean_{par.lower()}", f"geol_mean_{par.lower()}", "TOC_mg C/l", "hp"]

reg_df = reg_df[reg_df[f"{par}_µg/l"] < thresh].reset_index(drop=True)
utils.best_subsets_ols_regression(reg_df, resp_var, exp_vars, standardise=True)

### 7.2. Cr

In [ ]:
par = "Cr"

cols = [
    f"{par}_µg/l",
    f"moss_mean_{par.lower()}",
    f"geol_mean_{par.lower()}",
    "TOC_mg C/l",
    "hp",
]
reg_df = df[cols].copy()

with sn.plotting_context("notebook", font_scale=1.5):
    sn.pairplot(
        reg_df,
        kind="reg",
        diag_kind="kde",
        plot_kws=dict(lowess=True, line_kws=dict(color="red")),
    )

In [ ]:
thresh = 1
resp_var = f"{par}_µg/l"
exp_vars = [f"moss_mean_{par.lower()}", f"geol_mean_{par.lower()}", "TOC_mg C/l", "hp"]

reg_df = reg_df[reg_df[f"{par}_µg/l"] < thresh].reset_index(drop=True)
utils.best_subsets_ols_regression(reg_df, resp_var, exp_vars, standardise=True)

### 7.3. Cu

In [ ]:
par = "Cu"

cols = [
    f"{par}_µg/l",
    f"moss_mean_{par.lower()}",
    f"geol_mean_{par.lower()}",
    "TOC_mg C/l",
    "hp",
]
reg_df = df[cols].copy()

with sn.plotting_context("notebook", font_scale=1.5):
    sn.pairplot(
        reg_df,
        kind="reg",
        diag_kind="kde",
        plot_kws=dict(lowess=True, line_kws=dict(color="red")),
    )

In [ ]:
thresh = 7
resp_var = f"{par}_µg/l"
exp_vars = [f"moss_mean_{par.lower()}", f"geol_mean_{par.lower()}", "TOC_mg C/l", "hp"]

reg_df = reg_df[reg_df[f"{par}_µg/l"] < thresh].reset_index(drop=True)
utils.best_subsets_ols_regression(reg_df, resp_var, exp_vars, standardise=True)

### 7.4. Ni

In [ ]:
par = "Ni"

cols = [
    f"{par}_µg/l",
    f"moss_mean_{par.lower()}",
    f"geol_mean_{par.lower()}",
    "TOC_mg C/l",
    "hp",
]
reg_df = df[cols].copy()

with sn.plotting_context("notebook", font_scale=1.5):
    sn.pairplot(
        reg_df,
        kind="reg",
        diag_kind="kde",
        plot_kws=dict(lowess=True, line_kws=dict(color="red")),
    )

In [ ]:
thresh = 4
resp_var = f"{par}_µg/l"
exp_vars = [f"moss_mean_{par.lower()}", f"geol_mean_{par.lower()}", "TOC_mg C/l", "hp"]

reg_df = reg_df[reg_df[f"{par}_µg/l"] < thresh].reset_index(drop=True)
utils.best_subsets_ols_regression(reg_df, resp_var, exp_vars, standardise=True)

### 7.5. Pb

In [ ]:
par = "Pb"

cols = [
    f"{par}_µg/l",
    f"moss_mean_{par.lower()}",
    f"geol_mean_{par.lower()}",
    "TOC_mg C/l",
    "hp",
]
reg_df = df[cols].copy()

with sn.plotting_context("notebook", font_scale=1.5):
    sn.pairplot(
        reg_df,
        kind="reg",
        diag_kind="kde",
        plot_kws=dict(lowess=True, line_kws=dict(color="red")),
    )

In [ ]:
thresh = 5
resp_var = f"{par}_µg/l"
exp_vars = [f"moss_mean_{par.lower()}", f"geol_mean_{par.lower()}", "TOC_mg C/l", "hp"]

reg_df = reg_df[reg_df[f"{par}_µg/l"] < thresh].reset_index(drop=True)
utils.best_subsets_ols_regression(reg_df, resp_var, exp_vars, standardise=True)


png_path = r"../plots/metals/1k_lakes/ols_regression_pb.png"
plt.savefig(png_path, dpi=300, bbox_inches="tight")

### 7.6. Zn

In [ ]:
par = "Zn"

cols = [
    f"{par}_µg/l",
    f"moss_mean_{par.lower()}",
    f"geol_mean_{par.lower()}",
    "TOC_mg C/l",
    "hp",
]
reg_df = df[cols].copy()

with sn.plotting_context("notebook", font_scale=1.5):
    sn.pairplot(
        reg_df,
        kind="reg",
        diag_kind="kde",
        plot_kws=dict(lowess=True, line_kws=dict(color="red")),
    )

In [ ]:
thresh = 20
resp_var = f"{par}_µg/l"
exp_vars = [f"moss_mean_{par.lower()}", f"geol_mean_{par.lower()}", "TOC_mg C/l", "hp"]

reg_df = reg_df[reg_df[f"{par}_µg/l"] < thresh].reset_index(drop=True)
utils.best_subsets_ols_regression(reg_df, resp_var, exp_vars, standardise=True)

## 8. Summary

The relationships identified above are much more robust than those obtained using the 1995 1000 Lakes dataset: all relationships are highly statistically significant and most  are physically plausible. However, the explanatory power for several of the metals is poor (i.e. the effect size is small/negligible). Furthermore, in most cases the effects of **other water chemistry parameters (especially TOC concentration) are more important than either the moss or geochemistry variables**. This is problematic, because TEOTIL enforces a simple conceptual framework that is incapable of representing such "process-based" or "mechanistic" effects. 

**Note:** Similar problems have been documented previously for the original TEOTIL model when simulating phosphorus: because a significant fraction of the P-load is typically bound to particulates, the model performs poorly compared to nitrogen (where most of the flux is dissolved).

This notebook highlights the following problems/challenges with developing TEOTIL2 Metals:

 * The geochemistry dataset only includes values for **six out of eight** metals of interest
 
 * Although the moss suveys take place every five years, the number of data points has reduced substantially since 2005. Data for Hg are missing from the 1990 survey and, furthermore, in surveys prior to 2005 detection limits were substantially higher (similar to the 1995 1000 Lakes survey)
 
 * The statistical analysis required to identify export coefficients for the model identifies **significant but weak** relationships, where the best predictors are usually other water chemistry variables (pH and TOC) and **not** the moss or geochemistry datasets. Since we do not have national scale datasets of pH and TOC through time (the "1000 Lakes" are 20 years apart), it is not possible to use these relationships effectively. Furthermore, relationships restricted to using only the moss and geochemistry datasets are *very* poor
 
With these points in mind, **I do not think it is sensible to continue development of TEOTIL2 Metals in this direction**, at least for the moment. Instead, I will explore an alternative solution that makes better use of the 2019 1000 Lakes dataset.